In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import feature_manager as fma
import rl.env_simple_crypto_trade as env
import importlib
import rl.models as rla
import config as cf
import numpy as np
from torch.nn import Tanh
import tr_utils

In [ ]:
importlib.reload(fma)

fm = fma.FeatureManager()
fm.import_data(symbol="BTCUSDT",timeframes=["1d","1w","1mo"])

fm.build_features(
    lags = [1,1,1],
    features=cf.FOR_1D_INDICATORS,
)

In [2]:
%store -r fm

In [5]:
len(fm.cols)

125

In [6]:
importlib.reload(env)
importlib.reload(cf)

env_kwargs = cf.TRADE_ENV_PARAMETER
state_space = 10 + len(fm.cols)

full_env = env.CryptoTradingEnv(
    trade_timeframe="1d",
    df = fm.df, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)


In [10]:
importlib.reload(rla)
agent = rla.DRLTradeAgent(env=full_env)

catalog_name = "no_stop_loss_more_lags_4layers_1d_custom_ppo_gamma_0.9995_gae_lambda_0.95_n_steps_1536_ent_coef_0.001_learning_rate_0.00015_batch_size_64"

timestep = 420_000

selected_model = agent.load_model_from_checkpoint(
    "c_ppo",f"{catalog_name}/rl_model_{timestep}_steps")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Successfully load model from ../saved_models/checkpoint/no_stop_loss_more_lags_4layers_1d_custom_ppo_gamma_0.9995_gae_lambda_0.95_n_steps_1536_ent_coef_0.001_learning_rate_0.00015_batch_size_64/rl_model_420000_steps


In [12]:
agent

In [13]:
import wandb

In [14]:
wandb.init()
agent.predict(
    model=selected_model,
    environment=full_env,
    render=False
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anh-nguyendinh-cs. Use `wandb login --relogin` to force relogin


In [17]:
df = pd.DataFrame({"action":full_env.action_memory,"profit":full_env.trade_profit_memory})

In [21]:
df["position"] = df["action"].shift(-1)
open_position_df = df.loc[(df["position"]!=df["position"].shift()) & (df["position"]!=0)]
open_position_df.head()

,action,profit,position
0,1,-0.000200,1.0
26,1,0.000000,2.0
27,2,-0.332618,1.0
53,1,0.000000,2.0
54,2,-0.065547,1.0


In [26]:
df = pd.DataFrame({"action":full_env.action_memory,"profit":full_env.trade_profit_memory})
df["position"] = df["action"].shift(-1)
open_position_df = df.loc[(df["position"]!=df["position"].shift()) & (df["position"]!=0) ]
long_df = open_position_df.loc[df["position"]==1]
short_df = open_position_df.loc[df["position"]==2]
n_win_long = len(long_df[df["profit"]>0])
n_win_short  = len(short_df[df["profit"]>0])
long_win_rate = n_win_long/len(long_df) if n_win_long > 0  else 0
short_win_rate = n_win_short/len(short_df) if n_win_short > 0  else 0            
win_rate = (n_win_long+n_win_short)/(len(long_df)+len(short_df))

/tmp/ipykernel_1477700/4066109277.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  n_win_long = len(long_df[df["profit"]>0])
/tmp/ipykernel_1477700/4066109277.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  n_win_short  = len(short_df[df["profit"]>0])


In [ ]:
def func(x):
    return 1e-9
selected_model.set_env(full_env)
selected_model.lr_schedule = func
selected_model.learning_rate = 1e-9
selected_model.exploration_fraction= 0
selected_model.learning_starts=0
catalog_name = "dqn_4layers_relu_continue"

In [ ]:
selected_model.learn(
    total_timesteps = 5_000_000,
    tb_log_name = catalog_name,
    reset_num_timesteps = False,
    progress_bar = True,
)

In [ ]:
retrained_mode = agent.train_model(
    model = selected_model,
    total_timesteps = 5_000_000,
    checkpoint = True,
    catalog_name = catalog_name,
    save_frequency = 20_000,
    progress_bar = True
)

In [ ]:
agent.predict(
    model=selected_model,
    environment=full_env,
    render=False
)

In [ ]:
agent.describe_trades()

In [ ]:
agent.plot_results()

In [ ]:
agent.latest_model